In [7]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

lastException: Throwable = null


In [8]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [9]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [10]:
/*
inputDF.writeStream
      .format("console")
      .outputMode("append")
      .start()             // Start the computation
      .awaitTermination()  // Wait for the computation to terminate

*/

Name: Syntax Error.
Message: 
StackTrace: 

In [11]:
import java.time.Instant

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
//val getTimestamp = udf(() => Instant.now.getEpochSecond )

val loadedModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

//var predDF = Seq.empty[(String, Int, Long, Long)].toDF("tweet", "prediction", "clean_probability", "TS")

inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    
    val predictionsDF = loadedModel.transform(batchDF)
    //val now_minus_10 = getTimestamp() - 10
    val resDF = predictionsDF.select($"tweet", $"prediction", getProbability($"probability").alias("clean_probability")/*, getTimestamp().alias("TS")*/)
    resDF.show()
        
    //predDF = resDF.union(predDF)
    //resDF.show()
    //println(predDF.count()) //.filter($"TS" >= now_minus_10)    
    //println("За последние 10 секунд было спрогнозировано: ")
    //predDF.show(1000) //.filter($"TS" >= now_minus_10).groupBy($"prediction").count
}.start()

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
loadedModel = pipeline_0ce8a87d0323


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@18d18168

+--------------------+----------+-------------------+
|               tweet|prediction|  clean_probability|
+--------------------+----------+-------------------+
|just ate a bakewa...|       0.0|0.43810224662635244|
|i really fancy a ...|       0.0| 0.4999809859379349|
|man  practice sta...|       0.0| 0.4249372403219237|
|netball killed me...|       0.0|0.49057318846252435|
|just done bathing...|       1.0| 0.5115009639688723|
|well that went well |       1.0| 0.5115009639688723|
|I'm enjoying a qu...|       1.0| 0.5129109149108058|
|@therealpickler h...|       1.0| 0.5448869177916822|
|@heatheraewalker ...|       1.0| 0.5115009639688723|
|I wonder what Kri...|       1.0| 0.5233968235809713|
+--------------------+----------+-------------------+

+--------------------+----------+-------------------+
|               tweet|prediction|  clean_probability|
+--------------------+----------+-------------------+
|I'm sad to hear t...|       1.0| 0.5189018141259198|
|@johncmayer omg, ...|     